## Using LangChain to get structured outputs


In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_fireworks import ChatFireworks

from experiment_xml import (
    pydantic_to_xml_instructions,
    run_xml_experiment,
)

In [2]:
ANTHROPIC_API_KEY = "<API KEY>"
FIREWORKS_API_KEY = "<API KEY>"

In [3]:
import streamlit as st

LANGSMITH_API_KEY = st.secrets["api_keys"]["LANGSMITH_API_KEY"]
ANTHROPIC_API_KEY = st.secrets["api_keys"]["ANTHROPIC_API_KEY"]
FIREWORKS_API_KEY = st.secrets["api_keys"]["FIREWORKS_API_KEY"]

In [4]:
experiment_date = "24-02-25"
n_iter = 1

Let's start by creating a LLM model to run our structured output queries. Use a temperature of 0 to improve structured output generation (but at the cost of "creativity").


### Model setup


In [5]:
# LLM parameters
temperature = 0.8
timeout = 30
num_ctx = 8192
num_predict = 4096

In [6]:
llm_models_test = {
    "Ollama_llama32": ChatOllama(
        model="llama3.2",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "Ollama_phi3": ChatOllama(
        model="phi3",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "Ollama_deepseekr1": ChatOllama(
        model="deepseek-r1",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
}
llm_models = {
    "Ollama_llama32": ChatOllama(
        model="llama3.2",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "Ollama_nemotron": ChatOllama(
        model="nemotron-mini",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "Ollama_phi3": ChatOllama(
        model="phi3",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "Ollama_phi4": ChatOllama(
        model="phi4",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "Ollama_deepseekr1": ChatOllama(
        model="deepseek-r1",
        temperature=temperature,
        num_ctx=num_ctx,
        num_thread=1,
        num_predict=num_predict,
    ),
    "fireworks_llama31": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v3p1-70b-instruct",
        api_key=FIREWORKS_API_KEY,
        temperature=temperature,
        timeout=timeout,
    ),
    "fireworks_llama32": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v3p2-3b-instruct",
        api_key=FIREWORKS_API_KEY,
        temperature=temperature,
        timeout=timeout,
    ),
    "fireworks_llama33": ChatFireworks(
        model_name="accounts/fireworks/models/llama-v3p3-70b-instruct",
        api_key=FIREWORKS_API_KEY,
        temperature=temperature,
        timeout=timeout,
    ),
    # "fireworks_deepseekr1_70b": ChatFireworks(
    #     model_name="accounts/fireworks/models/deepseek-r1-distill-llama-70b",
    #     api_key=FIREWORKS_API_KEY,
    #     temperature=temperature,
    #     timeout=timeout,
    # ),
}
llm_models_with_anthropic = {
    **llm_models,
    "Anthropic_Sonnet_35": ChatAnthropic(
        model="claude-3-5-sonnet-20241022",
        api_key=ANTHROPIC_API_KEY,
        timeout=timeout,
    ),
    "Anthropic_Haiku_35": ChatAnthropic(
        model="claude-3-5-haiku-20241022",
        api_key=ANTHROPIC_API_KEY,
        timeout=timeout,
    ),
    "Anthropic_Haiku_3": ChatAnthropic(
        model="claude-3-haiku-20240307",
        api_key=ANTHROPIC_API_KEY,
        timeout=timeout,
    ),
}

### Problem setup and prompt


In [7]:
test_science_prompt_txt = """
You are a professional science writer tasked with responding to members of
the general public who write in asking questions about science.
Write an article responding to a writer's question for publication in a
science magazine intended for a general readership with a high-school education.
You should write clearly and compellingly, include all relavent context,
and provide motivating stories where applicable.

Your response must be less than 200 words.

The question given to you is the following:
{question}
"""

questions = [
    "What is the oldest recorded fossil?",
    "What is a black hole?",
    "How far away is the sun?",
    "Which other planet in the Solar System has a surface gravity closest to that of the Earth?",
    "Eris, Haumea, Makemake and Ceres are all examples of what?",
    "Why does earth have seasons? Do other planets exhibit seasons too?",
    "What causes the aurora borealis?",
    "Why is the sky blue?",
    "How do bees communicate?",
    "What is the smallest unit of life?",
    "How do plants make their own food?",
    "Why do we dream?",
    "What is the theory of relativity?",
    "How do volcanoes erupt?",
    "What is the speed of light?",
    "How do magnets work?",
    "What is the purpose of DNA?",
    "What are the different types of galaxies?",
    "Why do some animals hibernate?",
    "How do vaccines work?",
]

In [8]:
prompt_direct = ChatPromptTemplate.from_template(test_science_prompt_txt)

prompt_system_format = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Return a publishable article in the requested format.\n{format_instructions}",
        ),
        ("human", test_science_prompt_txt),
    ]
)

prompt_system_plus_reminder_format = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Return a publishable article in the requested format.\n{format_instructions}",
        ),
        (
            "human",
            test_science_prompt_txt + "\nYour response must be in valid XML.",
        ),
    ]
)

prompt_user_format = ChatPromptTemplate.from_template(
    test_science_prompt_txt + "\n{format_instructions}"
)

#### Schema using Pydantic XML


In [9]:
from pydantic_xml import BaseXmlModel, element, attr


class ArticleResponse1XML(BaseXmlModel, tag="article"):
    """Structured article for publication answering a reader's question."""

    title: str = element(description="Title of the article")
    answer: str = element(
        description="Provide a detailed description of historical events to answer the question"
    )
    number: int = element(description="A number that is most relevant to the question.")


class ArticleResponse1nointXML(BaseXmlModel, tag="article"):
    """Structured article for publication answering a reader's question."""

    title: str = element(description="Title of the article")
    answer: str = element(
        description="Provide a detailed description of historical events to answer the question"
    )
    number: str = element(description="A number that is most relevant to the question.")


# Lists of simple types
class ArticleResponse2XML(BaseXmlModel, tag="article"):
    """Structured article for publication answering a reader's question."""

    title: str = element(description="Title of the article")
    answer: str = element(description="Answer the writer's question")
    further_questions: list[str] = element(
        tag="further_question",
        description="A list of related questions that may be of interest to the readers.",
    )


# Nested types
class HistoricalEventXML(BaseXmlModel):
    """The year and explanation of a historical event."""

    year: str = element(description="The year of the historical event")
    event: str = element(
        description="A clear and concise explanation of what happened in this event"
    )


class ArticleResponse3XML(BaseXmlModel, tag="article"):
    """Structured article for publication answering a reader's question."""

    title: str = element(description="[Title of the article]")
    historical_event_1: HistoricalEventXML = element(
        description="A first historical event relevant to the question"
    )
    historical_event_2: HistoricalEventXML = element(
        description="A second historical event relevant to the question"
    )


# Lists of custom types
class ArticleResponse4XML(BaseXmlModel, tag="article"):
    """Structured article for publication answering a reader's question."""

    title: str = element(description="Title of the article")
    historical_timeline: list[HistoricalEventXML] = element(
        description="A list of historical events relevant to the question"
    )

In [10]:
structured_formats_xml = [
    dict(pydantic=schema, format_instructions=pydantic_to_xml_instructions(schema))
    for schema in [
        ArticleResponse1XML,
        ArticleResponse1nointXML,
        ArticleResponse2XML,
        ArticleResponse3XML,
        ArticleResponse4XML,
    ]
]

In [11]:
from langchain.output_parsers import XMLOutputParser

In [12]:
xml_output_parsers = [
    XMLOutputParser(name="article", tags=["article", "title", "answer", "number"]),
    XMLOutputParser(
        name="article", tags=["article", "title", "answer", "further_question"]
    ),
    XMLOutputParser(
        name="article",
        tags=[
            "article",
            "title",
            "historical_event_1",
            "year",
            "event",
            "historical_event_2",
            "year",
            "event",
        ],
    ),
    XMLOutputParser(
        name="article", tags=["article", "title", "historical_event", "year", "event"]
    ),
]
structured_formats_output_parser_xml = [
    dict(pydantic=schema, format_instructions=op.get_format_instructions())
    for schema, op in zip(
        [
            ArticleResponse1nointXML,
            ArticleResponse2XML,
            ArticleResponse3XML,
            ArticleResponse4XML,
        ],
        xml_output_parsers,
    )
]

Example prompt


In [13]:
print(pydantic_to_xml_instructions(ArticleResponse2XML))

You must respond only in XML using the following schema. Do not provide any explanation outside the XML.
<article>
  <!--Structured article for publication answering a reader's question.-->
  <title>
    {Title of the article - must be type str}
  </title>
  <answer>
    {Answer the writer's question - must be type str}
  </answer>
<!-- First list element -->
  <further_question>
    {A list of related questions that may be of interest to the readers. - must be type str}
  </further_question>
<!-- Next list element -->
  <further_question>
    {A list of related questions that may be of interest to the readers. - must be type str}
  </further_question>
<!-- Etc -->
  <further_question>
  ...
  </further_question>
</article>


### Evaluation


System prompt


In [14]:
if "structure_support_by_model_sp" not in locals():
    structure_support_by_model_sp = {}

_ = run_xml_experiment(
    prompt_system_format,
    questions,
    llm_models,
    structured_formats_xml,
    n_iter=1,
    results_out=structure_support_by_model_sp,
    save_file_name=f"exp5_xml_output_sys_{experiment_date}.pkl",
)

Model: Ollama_llama32  Output: ArticleResponse1XML   Pos: 1
Error: ValidationError
..Error: ValidationError
.Error: ValidationError
.Error: ValidationError
..Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
..Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.
Model: Ollama_llama32  Output: ArticleResponse1nointXML   Pos: 2
....................
Model: Ollama_llama32  Output: ArticleResponse2XML   Pos: 3
.........Error: XMLSyntaxError
....Error: XMLSyntaxError
.......
Model: Ollama_llama32  Output: ArticleResponse3XML   Pos: 4
Error: ValidationError
..Error: ValidationError
.Error: RuntimeError
.Error: RuntimeError
..Error: RuntimeError
......Error: RuntimeError
.Error: ValidationError
.Error: ValidationError
..Error: RuntimeError
.Error: ValidationError
..Error: XMLSyntaxError
.

Output parsers with system prompt


In [ ]:
if "structure_support_by_model_parsers" not in locals():
    structure_support_by_model_parsers = {}

_ = run_xml_experiment(
    prompt_system_format,
    questions,
    llm_models_with_anthropic,
    structured_formats_output_parser_xml,
    n_iter=1,
    results_out=structure_support_by_model_parsers,
    save_file_name=f"exp5_xml_output_parser_{experiment_date}.pkl",
)

Model: Ollama_llama32  Output: ArticleResponse1nointXML   Pos: 1
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: RuntimeError
.Error: XMLSyntaxError
.Error: ValidationError
..Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: XMLSyntaxError
.Error: XMLSyntaxError
.Error: RuntimeError
.Error: RuntimeError
.Error: RuntimeError
.Error: ValidationError
.Error: ValidationError
.
Model: Ollama_llama32  Output: ArticleResponse2XML   Pos: 2
.Error: ValidationError
.Error: RuntimeError
.Error: RuntimeError
..Error: XMLSyntaxError
....Error: XMLSyntaxError
.Error: RuntimeError
.Error: ValidationError
..Error: ValidationError
.Error: RuntimeError
.Error: ValidationError
.Error: RuntimeError
..Error: ValidationError
.Error: XMLSyntaxError
.
Model: Ollama_llama32  Output: ArticleResponse3XML   Pos: 3
Error: ValidationError
.Error: ValidationError
.Error: ValidationError
.Error: RuntimeError
.

Extract all error messages & count


In [18]:
import pandas as pd

In [ ]:
def results_to_df(ss_results, key="valid"):
    df = pd.DataFrame.from_dict(
        {
            mname: {
                tname: ss_results[mname][tname][key] * 100 / n_questions
                for tname in ss_results[mname].keys()
            }
            for mname in ss_results.keys()
        },
        orient="index",
    )
    return df


def analyse_errors_from_results(ss_results, method="code"):
    error_counts = {}
    for mname in ss_results.keys():
        error_counts[mname] = {}
        for tname in ss_results[mname].keys():
            # Count errors by failure code above
            if method == "code":
                error_types = pd.Series(
                    output["error_type"]
                    for output in ss_results[mname][tname]["outputs"]
                )
                error_codes = error_types.value_counts()

                for e_name, e_count in error_codes.items():
                    error_counts[mname][(tname, e_name)] = e_count

            elif method == "parse":
                # Count errors by parsing error message
                errors = (
                    output["error_message"]
                    for output in ss_results[mname][tname]["outputs"]
                )
                for error in errors:
                    if error is None:
                        continue
                    if error.lower().find("opening and ending tag mismatch") >= 0:
                        error_str = "XML tag mismatch"
                    elif error.lower().find("extracterror") >= 0:
                        error_str = "Missing main tags"
                    elif error.lower().find("input should be a valid integer") >= 0:
                        error_str = "Validation error (int)"
                    elif error.lower().find("premature end of data in tag") >= 0:
                        error_str = "Premature end"
                    elif error.lower().find("field required") >= 0:
                        error_str = "Missing field"
                    elif error.lower().find("expected '>'") >= 0:
                        error_str = "Tag malformed"
                    elif (
                        error.lower().find("extra content at the end of the document")
                        >= 0
                    ):
                        error_str = "Tag malformed"
                    elif error.find("XMLSyntaxError") >= 0:
                        error_str = "Other syntax error"
                    else:
                        error_str = error

                    error_counts[mname][(tname, error_str)] = (
                        error_counts[mname].get((tname, error_str), 0) + 1
                    )

            else:
                raise NameError(f"Method {method} not supported")

    return pd.DataFrame.from_dict(error_counts, orient="index")

In [ ]:
analyse_errors_from_results(
    structure_support_by_model_parsers, method="parse"
).sort_index(axis=1)

ArticleResponse1nointXML                    \
                             Missing field Missing main tags   
Ollama_llama32                          11                 4   
Ollama_phi3                              6                10   
Ollama_deepseekr1                        6                 8   

                                                      ArticleResponse2XML  \
                  Other syntax error XML tag mismatch       Missing field   
Ollama_llama32                   NaN                3                   5   
Ollama_phi3                      1.0                3                   4   
Ollama_deepseekr1                NaN                3                   6   

                                                                         \
                  Missing main tags Other syntax error XML tag mismatch   
Ollama_llama32                    5                NaN                3   
Ollama_phi3                       9                NaN                5   
Ollama_deepseekr1                 6                1.0                5   

                  ArticleResponse3XML                                       \
                        Missing field Missing main tags Other syntax error   
Ollama_llama32                     13                 6                NaN   
Ollama_phi3                         1                10                2.0   
Ollama_deepseekr1                   7                10                2.0   

                                                                \
                  Premature end Tag malformed XML tag mismatch   
Ollama_llama32              NaN           NaN              1.0   
Ollama_phi3                 NaN           1.0              6.0   
Ollama_deepseekr1           1.0           NaN              NaN   

                  ArticleResponse4XML                                       \
                        Missing field Missing main tags Other syntax error   
Ollama_llama32                     14                 5                NaN   
Ollama_phi3                         4                12                NaN   
Ollama_deepseekr1                   9                 5                2.0   

                                                  
                  Tag malformed XML tag mismatch  
Ollama_llama32              NaN                1  
Ollama_phi3                 1.0                3  
Ollama_deepseekr1           NaN                4

In [21]:
if 1:
    for output in structure_support_by_model_parsers["Ollama_llama32"][
        "ArticleResponse1nointXML"
    ]["outputs"]:
        if output["error_type"] != "ok":
            print(output["error_message"], "\n")
            print(output["raw"].content)

ValidationError, 1 validation error for ArticleResponse1nointXML
number
  [line 1]: Field required [type=missing, input_value={'title': 'Unraveling the...new discoveries.\n\n  '}, input_type=dict] 

<article>

  <title>Unraveling the Mystery of Black Holes</title>

  <answer>
    A black hole is one of the most mind-bending objects in the universe. It's essentially a region in space where the gravitational pull is so strong that nothing, not even light, can escape once it gets too close.

    Imagine a sinkhole that's so massive and dense that its gravity warps the fabric of spacetime around it. That's what a black hole is like. They're formed when a star collapses under its own weight, causing a massive amount of matter to be compressed into an incredibly small space.

    The term "black" doesn't mean they're dark or invisible – it actually comes from the way they absorb all light that gets too close. Scientists have observed many black holes using powerful telescopes and spacecraft,

In [184]:
results_list = {
    "System Prompt": structure_support_by_model_sp,
    "System + Reminder": structure_support_by_model_sprem,
    "User Prompt": structure_support_by_model_up,
    "Output Parsers": structure_support_by_model_parsers,
}

df_results = {}
for name, ss_results in results_list.items():
    df_results[name] = pd.DataFrame.from_dict(
        {
            tuple(mname.split("_")): {
                tname: ss_results[mname][tname]["valid"] * 100
                for tname in ss_results[mname].keys()
            }
            for mname in ss_results.keys()
        },
        orient="index",
    )
    display(name)

'System Prompt'

'System + Reminder'

'User Prompt'

'Output Parsers'

In [185]:
df = pd.concat(df_results).reorder_levels([1, 2, 0], axis=0).sort_index(axis=0)
df

ArticleResponse1XML  \
Ollama    deepseekr1 Output Parsers                     NaN   
                     System + Reminder                 75.0   
                     System Prompt                     80.0   
                     User Prompt                       90.0   
          llama32    Output Parsers                     NaN   
                     System + Reminder                 30.0   
                     System Prompt                     15.0   
                     User Prompt                       55.0   
          nemotron   System + Reminder                 50.0   
                     System Prompt                     15.0   
                     User Prompt                       25.0   
          phi3       Output Parsers                     NaN   
                     System + Reminder                 55.0   
                     System Prompt                     50.0   
                     User Prompt                       30.0   
          phi4       System + Reminder                 95.0   
                     System Prompt                     90.0   
                     User Prompt                      100.0   
fireworks llama31    System + Reminder                 70.0   
                     System Prompt                     75.0   
                     User Prompt                       85.0   
          llama32    System + Reminder                 35.0   
                     System Prompt                     40.0   
                     User Prompt                       45.0   
          llama33    System + Reminder                 95.0   
                     System Prompt                    100.0   
                     User Prompt                      100.0   

                                        ArticleResponse1nointXML  \
Ollama    deepseekr1 Output Parsers                         15.0   
                     System + Reminder                      85.0   
                     System Prompt                          80.0   
                     User Prompt                            95.0   
          llama32    Output Parsers                         10.0   
                     System + Reminder                      95.0   
                     System Prompt                         100.0   
                     User Prompt                           100.0   
          nemotron   System + Reminder                      75.0   
                     System Prompt                          25.0   
                     User Prompt                            20.0   
          phi3       Output Parsers                          0.0   
                     System + Reminder                      55.0   
                     System Prompt                          50.0   
                     User Prompt                            55.0   
          phi4       System + Reminder                     100.0   
                     System Prompt                         100.0   
                     User Prompt                           100.0   
fireworks llama31    System + Reminder                     100.0   
                     System Prompt                         100.0   
                     User Prompt                           100.0   
          llama32    System + Reminder                     100.0   
                     System Prompt                         100.0   
                     User Prompt                            95.0   
          llama33    System + Reminder                     100.0   
                     System Prompt                         100.0   
                     User Prompt                           100.0   

                                        ArticleResponse2XML  \
Ollama    deepseekr1 Output Parsers                    10.0   
                     System + Reminder                 85.0   
                     System Prompt                     65.0   
                     User Prompt                       70.0   
          llama32    Output Parsers                    

In [51]:
import tabulate

print(
    tabulate.tabulate(
        df.reset_index(), headers="keys", tablefmt="pipe", showindex=False
    )
)

| level_0   | level_1    | level_2           |   ArticleResponse1XML |   ArticleResponse1nointXML |   ArticleResponse2XML |   ArticleResponse3XML |   ArticleResponse4XML |
|:----------|:-----------|:------------------|----------------------:|---------------------------:|----------------------:|----------------------:|----------------------:|
| Ollama    | deepseekr1 | System + Reminder |                    75 |                         85 |                    85 |                    75 |                    45 |
| Ollama    | deepseekr1 | System Prompt     |                    80 |                         80 |                    65 |                    55 |                    75 |
| Ollama    | deepseekr1 | User Prompt       |                    90 |                         95 |                    70 |                    85 |                    80 |
| Ollama    | llama32    | System + Reminder |                    30 |                         95 |                    90 |                 

## Hypothesis testing


Hypothesis:

- H0: Results are the same
- H1: System prompt produces different results than user prompt for each method


In [55]:
structure_support_by_model_sp.keys()

dict_keys(['Ollama_llama32', 'Ollama_nemotron', 'Ollama_phi3', 'Ollama_phi4', 'Ollama_deepseekr1', 'fireworks_llama31', 'fireworks_llama32', 'fireworks_llama33'])

In [116]:
from scipy import stats

# Bonferroni correction
n_tests = len(structure_support_by_model_sp.keys())
alpha = 0.05 / n_tests


experiments_list = {
    "System Prompt": structure_support_by_model_sp,
    "System + Reminder": structure_support_by_model_sprem,
    # "User Prompt": structure_support_by_model_up,
}

for model in structure_support_by_model_sp.keys():

    contingency_table = {}
    for name, ss_results in experiments_list.items():
        num_true = 0
        num_total = 0
        for tname in ss_results[model].keys():
            num_true += ss_results[model][tname]["valid"] * len(questions)
            num_total += len(questions)

        contingency_table[name] = {"Passed": num_true, "Failed": num_total - num_true}

    ct = pd.DataFrame.from_dict(contingency_table, orient="index")

    print(f"\n{model}")
    print(sf := stats.fisher_exact(ct.to_numpy()))
    print(sb := stats.barnard_exact(ct.to_numpy()))
    if sf.pvalue < alpha:
        print(f"Hypothesis test passed: {sf.pvalue:.3g} < {alpha:.3g}")


Ollama_llama32
SignificanceResult(statistic=np.float64(0.8386446147640177), pvalue=np.float64(0.6566578617732227))
BarnardExactResult(statistic=np.float64(-0.592999453328881), pvalue=np.float64(0.6156492855579963))

Ollama_nemotron
SignificanceResult(statistic=np.float64(0.07258783663278046), pvalue=np.float64(1.0363568816780582e-14))
BarnardExactResult(statistic=np.float64(-7.6158294361716115), pvalue=np.float64(1.243989481529037e-14))
Hypothesis test passed: 1.04e-14 < 0.00625

Ollama_phi3
SignificanceResult(statistic=np.float64(0.5925925925925926), pvalue=np.float64(0.12422131626616004))
BarnardExactResult(statistic=np.float64(-1.6894111857387917), pvalue=np.float64(0.09520160147012026))

Ollama_phi4
SignificanceResult(statistic=np.float64(0.6598639455782312), pvalue=1.0)
BarnardExactResult(statistic=np.float64(-0.45291081365783814), pvalue=np.float64(0.9921818686071744))

Ollama_deepseekr1
SignificanceResult(statistic=np.float64(0.9055266887104393), pvalue=np.float64(0.87496546695

### Combined experiments : which model is better?


In [178]:
from scipy import stats

# Bonferroni correction
n_tests = len(structure_support_by_model_sp.keys())
alpha = 0.05 / n_tests


experiments_list = {
    "System Prompt": structure_support_by_model_sp,
    "System + Reminder": structure_support_by_model_sprem,
    "User Prompt": structure_support_by_model_up,
}

contingency_table = {}
for model in structure_support_by_model_sp.keys():

    num_true = 0
    num_total = 0
    for name, ss_results in experiments_list.items():
        for tname in ss_results[model].keys():
            # if tname == "ArticleResponse1XML":
            #     continue
            num_true += ss_results[model][tname]["valid"] * len(questions)
            num_total += len(questions)

    contingency_table[model] = {"Passed": num_true, "Failed": num_total - num_true}

ct = pd.DataFrame.from_dict(contingency_table, orient="index")

In [162]:
# Chi-Squared Test for multiple groups
res = stats.chi2_contingency(ct.T)
print(f"\nChi-Squared Test:\nStatistic={res.statistic:.4f}, p={res.pvalue:.4g}")


Chi-Squared Test:
Statistic=755.4006, p=7.781e-159


In [164]:
stats.fisher_exact(ct.T)

SignificanceResult(statistic=np.float64(2.0910616950343596e-177), pvalue=np.float64(0.0001))

### Pairwise tests between model and the best performance


In [165]:
best_performance = ct.iloc[ct.Passed.argmax()]
best_performance

Passed    239.0
Failed      1.0
Name: fireworks_llama31, dtype: float64

In [167]:
from scipy import stats

# Bonferroni correction
n_tests = len(structure_support_by_model_sp.keys())
alpha = 0.05 / n_tests

for model in structure_support_by_model_sp.keys():

    ct_2way = pd.concat([best_performance, ct.loc[model]], axis=1)

    print(f"\n{model}")
    print(sf := stats.fisher_exact(ct_2way.to_numpy()))
    print(sb := stats.barnard_exact(ct_2way.to_numpy()))
    if sf.pvalue < alpha:
        print(f"Hypothesis test passed: {sf.pvalue:.3g} < {alpha:.3g}")


Ollama_llama32
SignificanceResult(statistic=np.float64(55.15384615384615), pvalue=np.float64(1.5058467333595303e-13))
BarnardExactResult(statistic=np.float64(6.822593789375613), pvalue=np.float64(7.077856898151508e-12))
Hypothesis test passed: 1.51e-13 < 0.00625

Ollama_nemotron
SignificanceResult(statistic=np.float64(452.0843373493976), pvalue=np.float64(9.115512630681285e-63))
BarnardExactResult(statistic=np.float64(15.152655087714793), pvalue=np.float64(1.6165465666742128e-56))
Hypothesis test passed: 9.12e-63 < 0.00625

Ollama_phi3
SignificanceResult(statistic=np.float64(568.887323943662), pvalue=np.float64(4.504485863584975e-70))
BarnardExactResult(statistic=np.float64(16.033361802881117), pvalue=np.float64(3.0298231139859717e-64))
Hypothesis test passed: 4.5e-70 < 0.00625

Ollama_phi4
SignificanceResult(statistic=np.float64(3.0253164556962027), pvalue=np.float64(0.6234293142159372))
BarnardExactResult(statistic=np.float64(1.0041928905068658), pvalue=np.float64(0.3772075548532983